In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Join").getOrCreate()

In [3]:
spark

In [4]:
orders_rdd = spark.sparkContext.textFile("hdfs://localhost:9000/user/tkm/data/sample_superstore/orders.csv")

In [5]:
orders_rdd.take(3)

['Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit',
 '1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0,41.9136',
 '2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0,219.582']

In [14]:
returns_rdd =spark.sparkContext.textFile("hdfs://localhost:9000/user/tkm/data/sample_superstore/returns.csv")

In [7]:
returns_rdd.take(3)

['Returned,Order ID', 'Yes,CA-2017-153822', 'Yes,CA-2017-129707']

In [10]:
header_orders =  orders_rdd.first()
orders_filtered_rdd = orders_rdd.filter(lambda x : (x!=header_orders))
orders_filtered_rdd.take(3)

['1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0,41.9136',
 '2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0,219.582',
 '3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0,6.8714']

In [11]:
header_returns =  returns_rdd.first()
returns_filtered_rdd = returns_rdd.filter(lambda x : (x!=header_returns))
returns_filtered_rdd.take(3)

['Yes,CA-2017-153822', 'Yes,CA-2017-129707', 'Yes,CA-2014-152345']

In [12]:
orders_filtered_rdd.getNumPartitions()

2

In [13]:
returns_filtered_rdd.getNumPartitions()

2

In [17]:
mapped_orders_filtered_rdd = orders_filtered_rdd.map(lambda x : ((x.split(",")[1]),(x.split(",")[15])))

In [18]:
mapped_orders_filtered_rdd.take(3)

[('CA-2016-152156', 'Bookcases'),
 ('CA-2016-152156', 'Chairs'),
 ('CA-2016-138688', 'Labels')]

In [19]:
mapped_returns_filtered_rddd = returns_filtered_rdd.map(lambda x :((x.split(",")[1]),(x.split(",")[0])))

In [20]:
mapped_returns_filtered_rddd.take(3)

[('CA-2017-153822', 'Yes'),
 ('CA-2017-129707', 'Yes'),
 ('CA-2014-152345', 'Yes')]

In [21]:
joined_rdd = mapped_orders_filtered_rdd.join(mapped_returns_filtered_rddd)

In [22]:
joined_rdd.take(10)

[('CA-2014-143336', ('Art', 'Yes')),
 ('CA-2014-143336', ('Phones', 'Yes')),
 ('CA-2014-143336', ('Binders', 'Yes')),
 ('CA-2015-146262', ('Labels', 'Yes')),
 ('CA-2015-146262', ('Bookcases', 'Yes')),
 ('CA-2015-146262', ('Phones', 'Yes')),
 ('CA-2015-146262', ('Machines', 'Yes')),
 ('CA-2015-146262', ('Accessories', 'Yes')),
 ('CA-2016-148796', ('Chairs', 'Yes')),
 ('CA-2016-109869', ('Furnishings', 'Yes'))]

In [26]:
spark.stop()

In [28]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Join").getOrCreate()

orders_rdd = spark.sparkContext.textFile("hdfs://localhost:9000/user/tkm/data/sample_superstore/orders.csv")
returns_rdd =spark.sparkContext.textFile("hdfs://localhost:9000/user/tkm/data/sample_superstore/returns.csv")

header_orders =  orders_rdd.first()
header_returns =  returns_rdd.first() #these two are action but create a single default stage and 1 task  


#narrow transformation
orders_filtered_rdd = orders_rdd.filter(lambda x : (x!=header_orders))
returns_filtered_rdd = returns_rdd.filter(lambda x : (x!=header_returns)) #narrow transformation

#narrow transformation
mapped_orders_filtered_rdd = orders_filtered_rdd.map(lambda x : ((x.split(",")[1]),(x.split(",")[15])))
mapped_returns_filtered_rddd = returns_filtered_rdd.map(lambda x :((x.split(",")[1]),(x.split(",")[0])))


#wide transformation
joined_rdd = mapped_orders_filtered_rdd.join(mapped_returns_filtered_rddd)

#action
joined_rdd.saveAsTextFile("C:\\Users\\TARUN\\Desktop\\Pyspark\\superstore\\joined_results")